## Import Libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from faker import Faker
import random
np.random.seed()
import warnings
warnings.filterwarnings("ignore")

/tmp/ipykernel_348974/2556754689.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Create Dataset

In [4]:
x = []
palindromes = []
for i in range(1024):
    binary_string = format(i, '010b')
    x.append(binary_string)
    if binary_string == binary_string[::-1]:
        palindromes.append(binary_string)
x = np.array(x)
palindromes = np.array(palindromes)


In [5]:
print(len(palindromes))
palindromes

32


array(['0000000000', '0000110000', '0001001000', '0001111000',
       '0010000100', '0010110100', '0011001100', '0011111100',
       '0100000010', '0100110010', '0101001010', '0101111010',
       '0110000110', '0110110110', '0111001110', '0111111110',
       '1000000001', '1000110001', '1001001001', '1001111001',
       '1010000101', '1010110101', '1011001101', '1011111101',
       '1100000011', '1100110011', '1101001011', '1101111011',
       '1110000111', '1110110111', '1111001111', '1111111111'],
      dtype='<U10')

In [6]:
np.random.shuffle(palindromes)
print(len(palindromes))
palindromes

32


array(['1011001101', '1010000101', '1100110011', '0001111000',
       '0111111110', '0101111010', '0001001000', '1111001111',
       '0101001010', '0011001100', '1110000111', '0110000110',
       '1000000001', '1000110001', '0000110000', '1101111011',
       '0010000100', '0000000000', '1001111001', '0011111100',
       '0100110010', '1011111101', '1110110111', '0010110100',
       '1001001001', '1100000011', '1111111111', '0110110110',
       '0100000010', '0111001110', '1010110101', '1101001011'],
      dtype='<U10')

In [7]:
for i in range(5):
    x = np.concatenate((x, palindromes))
print(len(x))

1184


In [8]:
y = []
for binary_string in x:
    y.append(binary_string == binary_string[::-1])
y = np.array(y)
permutation_index = np.random.permutation(len(x))
x = x[permutation_index]
y = y[permutation_index]

In [9]:
def sigmoid(x):
    return 1 / (1 + np.exp(-1 * x))

def relu(x):
    return np.maximum(0, x)

In [10]:
# a = np.array([1, 2, -3])
# b = np.array([2, 3, 4])
# a*b

In [11]:
np.random.seed(42)
LEARNING_RATE = 1e-3
NUM_EPOCHS = 10000

input_size = 10
hidden_layer_size = 4
output_size = 1

weights_ih = np.random.rand(input_size, hidden_layer_size)
weights_ho = np.random.rand(hidden_layer_size, output_size)
bias_ih = np.random.rand(1, hidden_layer_size)
bias_ho = np.random.rand(1, output_size)

In [43]:
LEARNING_RATE = 0.5
NUM_EPOCHS = 100
for epoch in range(NUM_EPOCHS):
    tot_loss = 0
    
    for i, s in enumerate(x):
        inp = np.reshape(np.array([int(char) for char in s]), (1, input_size))
        # print(inp.shape)

        hlayer_logits = np.dot(inp, weights_ih) + bias_ih
        # print("hlayer_logits",hlayer_logits.shape)
        hlayer_output = sigmoid(hlayer_logits)
        # print("hlayer_output",hlayer_output.shape)

        final_logits = np.dot(hlayer_output, weights_ho) + bias_ho
        # print("final_logits",final_logits.shape)
        final_output = sigmoid(final_logits)


        tot_loss += abs(final_output[0] - y[i])

        output_delta = (y[i] - final_output[0]) * final_output * (1 - final_output[0])
        bias_ho += LEARNING_RATE * output_delta
        bias_ih += LEARNING_RATE * (output_delta.dot(weights_ho.T) * hlayer_output * (1 - hlayer_output))

        weights_ho += LEARNING_RATE * hlayer_output.T.dot(output_delta)
        weights_ih += LEARNING_RATE * inp.T.dot(output_delta.dot(weights_ho.T) * hlayer_output * (1 - hlayer_output))
        
    print(f'Epoch {epoch} : loss {tot_loss}')

Epoch 0 : loss [0.82641608]
Epoch 1 : loss [0.82609208]
Epoch 2 : loss [0.82576872]
Epoch 3 : loss [0.825446]
Epoch 4 : loss [0.82512391]
Epoch 5 : loss [0.82480246]
Epoch 6 : loss [0.82448164]
Epoch 7 : loss [0.82416145]
Epoch 8 : loss [0.82384189]
Epoch 9 : loss [0.82352295]
Epoch 10 : loss [0.82320464]
Epoch 11 : loss [0.82288695]
Epoch 12 : loss [0.82256987]
Epoch 13 : loss [0.82225341]
Epoch 14 : loss [0.82193756]
Epoch 15 : loss [0.82162233]
Epoch 16 : loss [0.8213077]
Epoch 17 : loss [0.82099368]
Epoch 18 : loss [0.82068026]
Epoch 19 : loss [0.82036745]
Epoch 20 : loss [0.82005523]
Epoch 21 : loss [0.81974362]
Epoch 22 : loss [0.81943259]
Epoch 23 : loss [0.81912216]
Epoch 24 : loss [0.81881233]
Epoch 25 : loss [0.81850308]
Epoch 26 : loss [0.81819441]
Epoch 27 : loss [0.81788633]
Epoch 28 : loss [0.81757884]
Epoch 29 : loss [0.81727192]
Epoch 30 : loss [0.81696558]
Epoch 31 : loss [0.81665982]
Epoch 32 : loss [0.81635463]
Epoch 33 : loss [0.81605002]
Epoch 34 : loss [0.81574597

In [44]:
print(bias_ho, bias_ih)

[[-14.98837889]] [[-0.24240941  2.22176463  1.49872937  2.03586551]]


In [45]:
x_test1 = '1100110011'
x_test2 = '1100110000'
def test(x):
    inp = np.reshape(np.array([int(char) for char in x]), (1, input_size))
    return sigmoid(np.dot(sigmoid(np.dot(inp, weights_ih)+ bias_ih), weights_ho)+bias_ho)

print(test(x_test1))
print(test(x_test2))

[[0.99999901]]
[[0.00016926]]


In [46]:
np.save('weights_ih.npy', weights_ih)
np.save('weights_ho.npy', weights_ho)
np.save('bias_ih', bias_ih)
np.save("bias_ho", bias_ho)

In [47]:
count=0
for i in range(len(x)):
    y_p = 1 if test(x[i])>0.5 else 0
    if y[i]!=y_p:
        count+=1
        print(x[i])

## New implementation

In [20]:
from utils import activation, dataset, losses
from model import *
import numpy as np

Dataset = dataset.Palindrome_Dataset(n_bits=10)
x_train,y_train = Dataset.get_data(shuffle=True, biasing_factor=0)

In [2]:
y_train[0]

1.0

In [3]:
x_temp = x_train[:4]
x_temp.shape

(4, 10)

In [4]:
model = Palindrome_Model(input_size=10, output_size=1,hidden_layer_sizes=[4],activation='sigmoid')
model.layers

[{'name': 'layer-1',
  'weights': array([[0.52104617, 0.07185501, 0.3294009 , 0.6100576 ],
         [0.57116313, 0.82440118, 0.8122118 , 0.50152226],
         [0.83367467, 0.0648887 , 0.2445651 , 0.78049713],
         [0.6211445 , 0.0257085 , 0.89328885, 0.16539725],
         [0.3090168 , 0.17966955, 0.43676407, 0.75964128],
         [0.81093867, 0.51780893, 0.71106392, 0.51255399],
         [0.93381801, 0.89909393, 0.38772025, 0.74320917],
         [0.5894768 , 0.95072858, 0.3834178 , 0.3517286 ],
         [0.76471038, 0.87794411, 0.69724492, 0.95569085],
         [0.36514249, 0.56562968, 0.45020004, 0.39085884]]),
  'biases': array([0.89168599, 0.91266535, 0.1714021 , 0.531869  ]),
  'activation': 'relu'},
 {'name': 'layer-2',
  'weights': array([[0.53269987],
         [0.66471686],
         [0.54252673],
         [0.53058214]]),
  'biases': array([0.3132721]),
  'activation': 'sigmoid'}]

In [5]:
model.forward(x_temp)

array([0.99991837, 0.99263631, 0.99996002, 0.99948973])

In [6]:
model.predict(x_temp)

array([1, 1, 1, 1])

In [18]:
model.set_optimizer(lr=100,loss="mse", optimizer="sgd")

In [21]:
accuracies, losses = model.train(x_train,y_train,epochs=200,batch_size=64)

Epoch 1/200:   0%|          | 0/1024 [00:00<?, ?sample/s]

Epoch 200/200:   0%|          | 1/1024 [00:00<00:53, 18.95sample/s, accuracy=96.9, loss=2.16e-5]


In [5]:
# y_pred = model(x)
# y_pred = np.clip(y_pred, 1e-12, 1 - 1e-12)
# -np.sum(y * np.log(y_pred) + (1 - y) * np.log(1 - (y_pred)))

NameError: name 'x' is not defined

In [7]:
temp_x = x[:48]
temp_y = y[:48]
print(temp_x, temp_y)

NameError: name 'x' is not defined

#### Resolving Bug

In [ ]:
from utils import dataset
from model import Palindrome_Model
import numpy as np
#np.random.seed(7)

Dataset = dataset.Palindrome_Dataset(n_bits=10)
x,y = Dataset.get_data(shuffle=True, biasing_factor=20)
model = Palindrome_Model(input_size=10, output_size=1,hidden_layer_sizes=[4],activation='sigmoid')
model.set_optimizer(lr=0.005,loss="bce", optimizer="sgd")

In [ ]:
temp = [np.array([1,0,1,1,1,1,1,1,0,1]), np.array([1,1,1,1,1,1,1,1,0,1]), np.array([0,0,0,0,1,1,0,0,0,0], dtype=float)]
y_temp = np.array([[1], [0], [1]])


for x,y in zip(temp,y_temp):
    print(x,y)

[1 0 1 1 1 1 1 1 0 1] [1]
[1 1 1 1 1 1 1 1 0 1] [0]
[0. 0. 0. 0. 1. 1. 0. 0. 0. 0.] [1]


In [ ]:
model.forward(temp)

array([[0.97403188],
       [0.91646151],
       [0.84928304]])

In [ ]:
model.backward(temp,y_temp)

Outputs [array([1, 0, 1, 1, 1, 1, 1, 1, 0, 1]), array([1.11147228, 0.        , 0.80965221, 2.29672724]), array([0.97403188])]
Label [1]
Predicted [0.97403188]
Loss Grad: [-1.02666045]
Act Grad: [0.02529378]
Loss Grad: [-0.026181    0.00943286  0.0268221  -0.04267106]
Act Grad: 1.0
Outputs [array([1, 1, 1, 1, 1, 1, 1, 1, 0, 1]), array([0.        , 0.        , 0.9913761 , 2.34475633]), array([0.91646151])]
Label [0]
Predicted [0.91646151]
Loss Grad: [11.97053005]
Act Grad: [0.07655981]
Loss Grad: [ 0.92397416 -0.33290256 -0.94659987  1.50593803]
Act Grad: 0.0
Outputs [array([0., 0., 0., 0., 1., 1., 0., 0., 0., 0.]), array([2.41982488, 1.27662388, 0.        , 0.11367569]), array([0.84928304])]
Label [1]
Predicted [0.84928304]
Loss Grad: [-1.17746376]
Act Grad: [0.12800136]
Loss Grad: [-0.15195245  0.05474759  0.15567337 -0.2476595 ]
Act Grad: 1.0


In [ ]:
np.array([-0.026181, 0.00943286,  0.0268221,  -0.04267106])*np.array([1.11147228, 0.        , 0.80965221, 2.29672724])

array([-0.02909946,  0.        ,  0.02171657, -0.09800379])

In [ ]:
np.outer(np.array([-0.026181, 0.00943286,  0.0268221,  -0.04267106]),np.array([1.11147228,      0.  , 0.80965221, 2.29672724]))

array([[-0.02909946, -0.        , -0.0211975 , -0.06013062],
       [ 0.01048436,  0.        ,  0.00763734,  0.02166471],
       [ 0.02981202,  0.        ,  0.02171657,  0.06160305],
       [-0.0474277 , -0.        , -0.03454872, -0.09800379]])

In [ ]:
model.weight_gradients

[array([[-0.008727  ,  0.00314429,  0.0089407 , -0.01422369],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [-0.008727  ,  0.00314429,  0.0089407 , -0.01422369],
        [-0.008727  ,  0.00314429,  0.0089407 , -0.01422369],
        [-0.05937782,  0.02139349,  0.06083182, -0.09677685],
        [-0.05937782,  0.02139349,  0.06083182, -0.09677685],
        [-0.008727  ,  0.00314429,  0.0089407 , -0.01422369],
        [-0.008727  ,  0.00314429,  0.0089407 , -0.01422369],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [-0.008727  ,  0.00314429,  0.0089407 , -0.01422369]]),
 array([[-0.1311905 ],
        [-0.06413629],
        [ 0.2958443 ],
        [ 0.69070146]])]

In [ ]:
model.bias_gradients

[array([-0.05937782,  0.02139349,  0.06083182, -0.09677685]),
 array([0.24659214])]

In [ ]:
x_palindromes = np.array([np.reshape(np.array([int(char) for char in s]), (10,)) for s in Dataset.palindromes])
y_palindromes = np.array([1]*32)

x_non_palindrome = np.array([np.reshape(np.array([int(char) for char in s]), (10,)) for s in Dataset.non_palindromes])
y_non_palindrome = np.array([0]*len(x_non_palindrome))
temp_x = np.concatenate((temp_x,x_palindromes))
temp_y = np.concatenate((temp_y, y_palindromes))
accuracies, losses = model.train(x_palindromes,y_palindromes,epochs=10)

In [ ]:
model.set_optimizer(lr=0.005,loss="mse")
accuracies, losses = model.train(temp_x,temp_y,epochs=100)

In [ ]:
predictions = model.predict(x_palindromes)
accuracy_metric(predictions, y_palindromes)


In [ ]:
predictions = model.predict(x_non_palindrome)
# print(predictions)
accuracy_metric(predictions, y_non_palindrome)

### Batch  Optimization

In [ ]:
model.layers

[{'name': 'layer-1',
  'weights': array([[ 0.93264673,  0.97459294, -0.12848894,  0.58921758],
         [-0.61967231, -0.26271703, -0.26330762, -0.93215773],
         [ 0.19383003, -1.06437594,  0.40447317, -0.24151142],
         [-0.44076791, -0.58080072, -0.21027712, -0.61963486],
         [-0.21196891, -0.25251799, -0.36971576, -0.07134193],
         [-0.01738654,  0.49040936, -0.42500098, -0.1270861 ],
         [-0.82970687,  0.77321702,  0.29156391, -0.54454293],
         [ 0.42967041, -0.02238592, -0.17803028,  0.43492721],
         [ 0.39676217, -0.09361975,  0.69012828,  0.31874703],
         [ 0.4822464 ,  0.3571908 ,  0.47563312,  0.07272139]]),
  'biases': array([-0.73642481, -0.58666945, -0.33631103,  0.80275304]),
  'activation': 'relu'},
 {'name': 'layer-2',
  'weights': array([[ 0.59679054],
         [-0.80576963],
         [ 0.08172214],
         [-0.04323667]]),
  'biases': array([2.17619008]),
  'activation': 'sigmoid'}]

[1 0 1 1 1 1 1 1 0 1] 1
[1 1 1 1 1 1 1 1 0 1] 0


In [ ]:
model.forward(temp)

array([[0.88995978],
       [0.89793558]])

### K-Fold Cross Validation

In [ ]:
def k_fold_cross_validation(model,X:np.ndarray, y:np.ndarray, k_folds:int = 4, epochs_per_fold:int = 25):
        fold_size = len(X) // k_folds
        accuracies_fold = []
        accuracies=[]
        losses = []
        for i in range(k_folds):
            start, end = i * fold_size, (i + 1) * fold_size
            X_test_fold, y_test_fold = X[start:end], y[start:end]
            X_train_fold = np.concatenate([X[:start], X[end:]])
            y_train_fold = np.concatenate([y[:start], y[end:]])
            print(f"Training Fold {i+1}\n")
            accuracy, loss = model.train(X_train_fold, y_train_fold, epochs_per_fold)
            accuracies.extend(accuracy)
            losses.extend(loss)
            print("\n"+10*"----"+"\n")
            predictions = model.predict(X_test_fold)
            accuracies_fold.append(accuracy_metric(predictions, y_test_fold))
        return accuracies_fold

In [ ]:
accs = k_fold_cross_validation(model, x, y, epochs_per_fold=50)

In [ ]:
np.average(accs)

In [ ]:
accs

In [ ]:
predictions = model.predict(x)
accuracy_metric(predictions, y)

In [ ]:
model.layers

In [ ]:
count = 0
for i in range(len(y)):
    if predictions[i]==1:
        count+=1
        # print(x[i], y[i], predictions[i])
count

In [ ]:
count

In [ ]:
len(x)

1664

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the model
model = models.Sequential([
    layers.InputLayer(input_shape=(10,)),          # Input layer with size 10
    layers.Dense(4, activation='relu'),          # Hidden layer with size 4 and linear activation
    layers.Dense(1, activation='sigmoid')           # Output layer with size 1 and sigmoid activation
])

optimizer = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.0)
# Compile the model
model.compile(optimizer=optimizer,loss='mse', metrics=['accuracy'])

# Display the model summary
model.summary()

2024-02-13 23:04:12.338627: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-13 23:04:12.341532: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-13 23:04:12.378010: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-13 23:04:12.378049: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-13 23:04:12.378944: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 44        
                                                                 
 dense_1 (Dense)             (None, 1)                 5         
                                                                 
Total params: 49 (196.00 Byte)
Trainable params: 49 (196.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


2024-02-13 23:04:14.371571: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-13 23:04:14.371946: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
model.fit(x_train, y_train, epochs=100, batch_size=8)

Epoch 1/100
208/208 [==============================] - 0s 944us/step - loss: 0.2419 - accuracy: 0.5980
Epoch 2/100
208/208 [==============================] - 0s 928us/step - loss: 0.2322 - accuracy: 0.6250
Epoch 3/100
208/208 [==============================] - 0s 873us/step - loss: 0.2205 - accuracy: 0.6508
Epoch 4/100
208/208 [==============================] - 0s 903us/step - loss: 0.2045 - accuracy: 0.6701
Epoch 5/100
208/208 [==============================] - 0s 1ms/step - loss: 0.1936 - accuracy: 0.6911
Epoch 6/100
208/208 [==============================] - 0s 957us/step - loss: 0.1863 - accuracy: 0.7206
Epoch 7/100
208/208 [==============================] - 0s 976us/step - loss: 0.1821 - accuracy: 0.7121
Epoch 8/100
208/208 [==============================] - 0s 955us/step - loss: 0.1782 - accuracy: 0.7236
Epoch 9/100
208/208 [==============================] - 0s 886us/step - loss: 0.1758 - accuracy: 0.7236
Epoch 10/100
208/208 [==============================] - 0s 915us/step - los

In [ ]:
x

In [ ]:
y

In [ ]:
def is_palindrome(arr):
    if np.array_equal(arr, np.flip(arr)):
        return 1
    else:
        return 0

In [ ]:
y_pred = []
for inp in x:
    y_pred.append(is_palindrome(inp))

count = 0
for i in range(len(y)):
    if not y_pred[i]==y[i]:
        count+=1

In [ ]:
count